In [5]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore

# Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for 
# more details on how to retrieve for all entities in the offline store instead
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2023, 4, 12, 10, 59, 42),
            datetime(2023, 4, 12, 8, 12, 10),
            datetime(2023, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)


store = FeatureStore(repo_path="./feast_project/feature_repo")
entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)
feature_service = store.get_feature_service("driver_activity_v1")


training_df = store.get_historical_features(
    entity_df=entity_df,features=feature_service,
    # features=[
    #     "driver_hourly_stats:conv_rate",
    #     "driver_hourly_stats:acc_rate",
    #     "driver_hourly_stats:avg_daily_trips",
    # ],
).to_df()

print("----- Feature schema -----\n")
print(training_df.info())

print()
print("----- Example features -----\n")
print(training_df.head())

/home/cdsw/.local/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:67: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  warnings.warn(
/home/cdsw/.local/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:137: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/home/cdsw/.local/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .item

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   driver_id                           3 non-null      int64         
 1   event_timestamp                     3 non-null      datetime64[ns]
 2   label_driver_reported_satisfaction  3 non-null      int64         
 3   val_to_add                          3 non-null      int64         
 4   val_to_add_2                        3 non-null      int64         
 5   conv_rate                           3 non-null      float32       
 6   conv_rate_plus_val1                 3 non-null      float64       
 7   conv_rate_plus_val2                 3 non-null      float64       
dtypes: datetime64[ns](1), float32(1), float64(2), int64(4)
memory usage: 308.0 bytes
None

----- Example features -----

   driver_id            event_

/home/cdsw/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/cdsw/.local/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/cdsw/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [6]:
training_df

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,conv_rate_plus_val1,conv_rate_plus_val2
0,1003,2024-02-23 05:35:37.204273,3,3,30,0.745193,3.745193,30.745193
1,1001,2024-02-23 05:35:37.204273,1,1,10,0.812710,1.812710,10.812710
2,1002,2024-02-23 05:35:37.204273,5,2,20,0.131389,2.131389,20.131389


In [7]:
%%bash
#ingest data into Online store 
cd feast_project/feature_repo
CURRENT_TIME=$(date -u +"%Y-%m-%dT%H:%M:%S")
feast materialize-incremental $CURRENT_TIME

/home/cdsw/.local/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark_source.py:67: RuntimeWarning: The spark data source API is an experimental feature in alpha development. This API is unstable and it could and most probably will be changed in the future.
  warnings.warn(


Materializing 2 feature views to 2024-02-23 05:37:02+00:00 into the sqlite online store.

driver_hourly_stats_fresh from 2022-10-11 05:37:09+00:00 to 2024-02-23 05:37:02+00:00:


02/23/2024 05:37:09 AM urllib3.connection WARNING: Certificate did not match expected hostname: usage.feast.dev. Certificate: {'subject': ((('countryName', 'US'),), (('stateOrProvinceName', 'California'),), (('localityName', 'San Francisco'),), (('organizationName', 'Netlify, Inc'),), (('commonName', '*.netlify.app'),)), 'issuer': ((('countryName', 'US'),), (('organizationName', 'DigiCert Inc'),), (('commonName', 'DigiCert Global G2 TLS RSA SHA256 2020 CA1'),)), 'version': 3, 'serialNumber': '03ED167A2659DA278E7B21CDBEB33E32', 'notBefore': 'Jan 15 00:00:00 2024 GMT', 'notAfter': 'Feb 14 23:59:59 2025 GMT', 'subjectAltName': (('DNS', '*.netlify.app'), ('DNS', 'netlify.app')), 'OCSP': ('http://ocsp.digicert.com',), 'caIssuers': ('http://cacerts.digicert.com/DigiCertGlobalG2TLSRSASHA2562020CA1-1.crt',), 'crlDistributionPoints': ('http://crl3.digicert.com/DigiCertGlobalG2TLSRSASHA2562020CA1-1.crl', 'http://crl4.digicert.com/DigiCertGlobalG2TLSRSASHA2562020CA1-1.crl')}
Using Spark's default

Pulling latest features from spark offline store


24/02/23 05:37:13 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 936fbc71-f1da-47dd-a138-f4d1a5019565
24/02/23 05:37:21 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
/home/cdsw/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/cdsw/.local/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/cdsw/.local/lib/python3.9/site-packages/pyspa

driver_hourly_stats from 2022-10-11 05:37:29+00:00 to 2024-02-23 05:37:02+00:00:


/home/cdsw/.local/lib/python3.9/site-packages/feast/infra/offline_stores/contrib/spark_offline_store/spark.py:78: RuntimeWarning: The spark offline store is an experimental feature in alpha development. Some functionality may still be unstable so functionality can change in the future.
  warnings.warn(


Pulling latest features from spark offline store


/home/cdsw/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/home/cdsw/.local/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/home/cdsw/.local/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:194: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
100%|█████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 36.02it/s]
/home/cdsw/.local/lib/pytho

In [9]:
# Fetching Feature vectors for inference
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path="./feast_project/feature_repo")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1001, "val_to_add": 1000, "val_to_add_2": 2000,},
        {"driver_id": 1002, "val_to_add": 3000,"val_to_add_2": 4000,},
    ],
).to_dict()

pprint(feature_vector)


{'acc_rate': [0.011025987565517426, 0.3090273141860962],
 'avg_daily_trips': [711, 44],
 'conv_rate': [0.8127095699310303, 0.13138850033283234],
 'driver_id': [1001, 1002]}


In [13]:
#USING FEATURE SERVICE TO FETCH ONLINE FEATURES

from pprint import pprint
from feast import FeatureStore

feature_store = FeatureStore(repo_path="./feast_project/feature_repo")

feature_service = feature_store.get_feature_service("driver_activity_v1")
feature_vector = feature_store.get_online_features(
    features=feature_service,
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1001, "val_to_add": 1000, "val_to_add_2": 2000,},
        {"driver_id": 1002, "val_to_add": 3000,"val_to_add_2": 4000,},
    ],
).to_dict()
pprint(feature_vector)

{'conv_rate': [0.8127095699310303, 0.13138850033283234],
 'conv_rate_plus_val1': [1000.812709569931, 3000.131388500333],
 'conv_rate_plus_val2': [2000.812709569931, 4000.131388500333],
 'driver_id': [1001, 1002]}


/home/cdsw/.local/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 47618)
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/ho

In [ ]:
%%bash
#Step 4: Step 4: Browse your features with the Web UI (experimental)
cd good_mallard
feast ui